In [1]:
import numpy as np
import pandas as pd

In [2]:
with open('./eval.txt', 'r') as f:
    results = f.readlines()

# Remove any non-result lines from the eval file, and split the lines on the tab character
# (results have format: model_name\tdataset_name\tmetric_name\tmetric_value)
results = [r.replace('\n','').split('\t') for r in results if '\t' in r]

In [3]:
def remove_underscore_after(val): # Remove underscore, and keep the part after the underscore
    return val.split('_')[-1]
def remove_underscore_before(val): # Remove underscore, and keep the part before the underscore
    return val.split('_')[0]
def map2d(func, grid): # Mapping for 2d arrays, from: https://stackoverflow.com/questions/70742445/elegant-map-over-2d-list
    return [[func(value) for value in row] for row in grid]
def full_display(df):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        display(df)

df = pd.DataFrame(results, columns = ['model', 'dataset', 'metric', 'value'])
df['value'] = pd.to_numeric(df['value'])
df['model'] = df['model'].map(lambda x: '-'.join(x.split('-')[2:]))
models = df['model'].str.split('-').tolist()

# Remove all underscores from our 2d list, keep one list of the param names and one with param vals
model_names_list = map2d(remove_underscore_before, models)
model_names = np.array(model_names_list)
model_val_list = map2d(remove_underscore_after, models)
model_vals = np.array(model_val_list)

In [4]:
model_vals.shape, model_names.shape

((41630, 14), (41630, 14))

In [5]:
# Splitting model name into more columns
for i in range(model_names.shape[-1]):
    name = model_names[0][i]
    val = model_vals[:,i]
    df[name] = val
    try:
       df[name] = pd.to_numeric(df[name]) 
    except:
        pass

# Remove columns that aren't relevant
df = df.drop(['fold', 'vit', 'model', 'data', 'ALL',  'kw'], axis = 1) #'method', 'AL.iter', 'ratio', 'PL',

# Replace 'None' with NaN, to allow conversion to numerical
df['AL.iter'] = df['AL.iter'].replace('None', np.nan)
df['AL.iter'] = pd.to_numeric(df['AL.iter'])
df['AL.epochs'] = df['AL.epochs'].replace('None', np.nan)
df['AL.epochs'] = pd.to_numeric(df['AL.epochs'])

cols = sorted(df.columns.tolist())
print(cols)
# The ratio have the format: 'ratio_xxx' where xxx is some float.
# We only want that float value, and need to convert it from string to float.
# df['ratio'] = pd.to_numeric(df['ratio'])
display(df)

# Group by all columns except the performance values, then compute mean, std and number of model runs for the performance
df_grouped = df.groupby(list(set(cols)-set(['value'])), dropna = False).agg({'value':['mean', 'std', 'count']})

['AL.epochs', 'AL.iter', 'PL', 'bs', 'dataset', 'epochs', 'lr', 'method', 'metric', 'ratio', 'value']


,dataset,metric,value,ratio,method,AL.iter,AL.epochs,PL,epochs,lr,bs
0,RSICD-CLS,zeroshot-val-top1,60.99,0.1,base,NaN,NaN,None,5,0.000050,64
1,UCM-CLS,zeroshot-val-top1,56.84,0.1,base,NaN,NaN,None,5,0.000050,64
2,RSICD,image_to_text_R@1,4.76,0.1,base,NaN,NaN,None,5,0.000050,64
3,RSICD,image_to_text_R@5,20.05,0.1,base,NaN,NaN,None,5,0.000050,64
4,RSICD,image_to_text_R@10,33.88,0.1,base,NaN,NaN,None,5,0.000050,64
...,...,...,...,...,...,...,...,...,...,...,...
41625,Sydney,image_to_text_R@5,11.07,0.1,base,NaN,NaN,None,35,0.000005,128
41626,Sydney,image_to_text_R@10,18.51,0.1,base,NaN,NaN,None,35,0.000005,128
41627,Sydney,text_to_image_R@1,2.82,0.1,base,NaN,NaN,None,35,0.000005,128
41628,Sydney,text_to_image_R@5,12.07,0.1,base,NaN,NaN,None,35,0.000005,128


In [6]:
display(df_grouped)

value  \
                                                                                             mean   
metric            dataset AL.epochs bs  method ratio PL        epochs AL.iter lr                    
image_to_text_R@1 RSICD   1.0       64  base   0.1   None      1      3.0     0.000010   1.100000   
                          5.0       64  base   0.1   None      20     3.0     0.000010   3.570000   
                                                                      5.0     0.000010   3.570000   
                                                                      7.0     0.000010   1.830000   
                          10.0      64  base   0.1   None      10     3.0     0.000010   2.700000   
...                                                                                           ...   
zeroshot-val-top1 UCM-CLS NaN       128 ours   0.1   soft.text 30     NaN     0.000050  68.125000   
                                                                              0.000500   4.761111   
                                                               35     NaN     0.000005  56.900000   
                                                                              0.000050  67.912500   
                                                                              0.000500   4.808889   

                                                                                                  \
                                                                                             std   
metric            dataset AL.epochs bs  method ratio PL        epochs AL.iter lr                   
image_to_text_R@1 RSICD   1.0       64  base   0.1   None      1      3.0     0.000010       NaN   
                          5.0       64  base   0.1   None      20     3.0     0.000010       NaN   
                                                                      5.0     0.000010       NaN   
                                                                      7.0     0.000010       NaN   
                          10.0      64  base   0.1   None      10     3.0     0.000010  1.230366   
...                                                                                          ...   
zeroshot-val-top1 UCM-CLS NaN       128 ours   0.1   soft.text 30     NaN     0.000050  1.641595   
                                                                              0.000500  0.255805   
                                                               35     NaN     0.000005  1.654630   
                                                                              0.000050  2.067646   
                                                                              0.000500  0.187979   

                                                                                              
                                                                                       count  
metric            dataset AL.epochs bs  method ratio PL        epochs AL.iter lr              
image_to_text_R@1 RSICD   1.0       64  base   0.1   None      1      3.0     0.000010     1  
                          5.0       64  base   0.1   None      20     3.0     0.000010     1  
                                                                      5.0     0.000010     1  
                                                                      7.0     0.000010     1  
                          10.0      64  base   0.1   None      10     3.0     0.000010     2  
...                                                                                      ...  
zeroshot-val-top1 UCM-CLS NaN       128 ours   0.1   soft.text 30     NaN     0.000050     4  
                                                                              0.000500     9  
                                                               35     NaN     0.000005     4  
                                                                              0.000050     4  
                                         

In [7]:
df_grouped.to_csv('eval.csv')

In [8]:
df_results = df_grouped.reset_index()
df_results['AL.epochs'] = df_results['AL.epochs'].fillna(df_results['epochs'])
df_results = df_results.sort_values(['metric', 'dataset', ('value', 'mean')])
# df_results = df_results.groupby(['metric', 'dataset'])
# df_results = df_grouped.reset_index()
# df_results = df_results.sort_values(['metric', 'dataset', ('value', 'mean')])

# df_2 = df_2[(df_2['method'] == 'ours') & (df_2['ratio'] == 0.2) & (df_2['PL'] == 'ot.image')] # (df_2['metric'] == 'image_to_text_R@5') &
# df_2 = df_2[(df_2['epochs'] > 20)  & (df_2['bs'] == 64) & (df_2['lr']==5e-5)] # &(df_2['lr']==5e-5)
# df_2 = df_2[(df_2['lr']==5e-5) & (df_2['bs'] == 64) & (df_2['method'] == 'base')] # &(df_2['lr']==5e-5)
display(df_results)

metric  dataset AL.epochs   bs method ratio         PL  \
                                                                          
251   image_to_text_R@1    RSICD      35.0  128   ours   0.1  hard.text   
170   image_to_text_R@1    RSICD       5.0   64   ours   0.1  soft.text   
185   image_to_text_R@1    RSICD      30.0   64   ours   0.1  soft.text   
107   image_to_text_R@1    RSICD       5.0   64   ours   0.1   ot.image   
119   image_to_text_R@1    RSICD      25.0   64   ours   0.1   ot.image   
...                 ...      ...       ...  ...    ...   ...        ...   
6441  zeroshot-val-top1  UCM-CLS       5.0   64   base   0.5       None   
6446  zeroshot-val-top1  UCM-CLS      15.0   64   base   0.7       None   
6444  zeroshot-val-top1  UCM-CLS       5.0   64   base   0.7       None   
6443  zeroshot-val-top1  UCM-CLS      15.0   64   base   0.5       None   
6445  zeroshot-val-top1  UCM-CLS      10.0   64   base   0.7       None   

     epochs AL.iter       lr    value                  
                                 mean       std count  
251      35     NaN  0.00050   0.0450  0.051962     4  
170       5     NaN  0.00050   0.0675  0.045000     4  
185      30     NaN  0.00050   0.0675  0.045000     4  
107       5     NaN  0.00050   0.0700  0.060000     9  
119      25     NaN  0.00050   0.0800  0.030000     9  
...     ...     ...      ...      ...       ...   ...  
6441      5     NaN  0.00005  77.7550  3.436680     4  
6446     15     NaN  0.00005  77.9725  3.209884     4  
6444      5     NaN  0.00005  78.1875  5.256433     4  
6443     15     NaN  0.00005  78.2875  3.939225     4  
6445     10     NaN  0.00005  78.3975  3.115032     4  

[6720 rows x 13 columns]

In [9]:
df_results.shape

(6720, 13)

In [10]:
# Results with basic active learning
df_AL_basic = df_results[(df_results['AL.iter']>0) & (df_results['epochs']==15) & (df_results['ratio'] == 0.1)]
full_display(df_AL_basic.groupby(['metric', 'dataset']).tail(3)) #

metric    dataset AL.epochs  bs method ratio    PL epochs  \
                                                                              
30     image_to_text_R@1      RSICD      15.0  64   base   0.1  None     15   
32     image_to_text_R@1      RSICD      15.0  64   base   0.1  None     15   
31     image_to_text_R@1      RSICD      15.0  64   base   0.1  None     15   
347    image_to_text_R@1     Sydney      15.0  64   base   0.1  None     15   
368    image_to_text_R@1     Sydney      15.0  64   base   0.1  None     15   
366    image_to_text_R@1     Sydney      15.0  64   base   0.1  None     15   
703    image_to_text_R@1        UCM      15.0  64   base   0.1  None     15   
704    image_to_text_R@1        UCM      15.0  64   base   0.1  None     15   
683    image_to_text_R@1        UCM      15.0  64   base   0.1  None     15   
1019  image_to_text_R@10      RSICD      15.0  64   base   0.1  None     15   
1039  image_to_text_R@10      RSICD      15.0  64   base   0.1  None     15   
1040  image_to_text_R@10      RSICD      15.0  64   base   0.1  None     15   
1355  image_to_text_R@10     Sydney      15.0  64   base   0.1  None     15   
1376  image_to_text_R@10     Sydney      15.0  64   base   0.1  None     15   
1374  image_to_text_R@10     Sydney      15.0  64   base   0.1  None     15   
1711  image_to_text_R@10        UCM      15.0  64   base   0.1  None     15   
1691  image_to_text_R@10        UCM      15.0  64   base   0.1  None     15   
1712  image_to_text_R@10        UCM      15.0  64   base   0.1  None     15   
2027   image_to_text_R@5      RSICD      15.0  64   base   0.1  None     15   
2048   image_to_text_R@5      RSICD      15.0  64   base   0.1  None     15   
2047   image_to_text_R@5      RSICD      15.0  64   base   0.1  None     15   
2363   image_to_text_R@5     Sydney      15.0  64   base   0.1  None     15   
2384   image_to_text_R@5     Sydney      15.0  64   base   0.1  None     15   
2382   image_to_text_R@5     Sydney      15.0  64   base   0.1  None     15   
2719   image_to_text_R@5        UCM      15.0  64   base   0.1  None     15   
2720   image_to_text_R@5        UCM      15.0  64   base   0.1  None     15   
2699   image_to_text_R@5        UCM      15.0  64   base   0.1  None     15   
3035   text_to_image_R@1      RSICD      15.0  64   base   0.1  None     15   
3055   text_to_image_R@1      RSICD      15.0  64   base   0.1  None     15   
3056   text_to_image_R@1      RSICD      15.0  64   base   0.1  None     15   
3371   text_to_image_R@1     Sydney      15.0  64   base   0.1  None     15   
3390   text_to_image_R@1     Sydney      15.0  64   base   0.1  None     15   
3392   text_to_image_R@1     Sydney      15.0  64   base   0.1  None     15   
3727   text_to_image_R@1        UCM      15.0  64   base   0.1  None     15   
3707   text_to_image_R@1        UCM      15.0  64   base   0.1  None     15   
3728   text_to_image_R@1        UCM      15.0  64   base   0.1  None     15   
4043  text_to_image_R@10      RSICD      15.0  64   base   0.1  None     15   
4063  text_to_image_R@10      RSICD      15.0  64   base   0.1  None     15   
4064  text_to_image_R@10      RSICD      15.0  64   base   0.1  None     15   
4399  text_to_image_R@10     Sydney      15.0  64   base   0.1  None     15   
4398  text_to_image_R@10     Sydney      15.0  64   base   0.1  None     15   
4400  text_to_image_R@10     Sydney      15.0  64   base   0.1  None     15   
4735  text_to_image_R@10        UCM      15.0  64   base   0.1  None     15   
4715  text_to_image_R@10        UCM      15.0  64   base   0.1  None     15   
4736  text_to_image_R@10        UCM      15.0  64   base   0.1  None     15   
5051   text_to_image_R@5      RSICD      15.0  64   base   0.1  None     15   
5071   text_to_image_R@5      RSICD      15.0  64   base   0.1  None     15   
5072   text_to_image_R@5      RSICD      15.0  64   base   0.1  None     15   
5407   text_to_image_R@5     Sydney      15.0  64   base   0.1  None

In [11]:
# Results for base CLIP model (filtered a bit)
df_baseline = df_results[(df_results['AL.iter'].isna()) & (df_results['method'] == 'base') & (df_results['ratio']==0.1)]
full_display(df_baseline.groupby(['metric', 'dataset']).tail(3))

metric    dataset AL.epochs   bs method ratio    PL epochs  \
                                                                               
46     image_to_text_R@1      RSICD      25.0   64   base   0.1  None     25   
52     image_to_text_R@1      RSICD      35.0   64   base   0.1  None     35   
49     image_to_text_R@1      RSICD      30.0   64   base   0.1  None     30   
385    image_to_text_R@1     Sydney      30.0   64   base   0.1  None     30   
541    image_to_text_R@1     Sydney      30.0  128   base   0.1  None     30   
544    image_to_text_R@1     Sydney      35.0  128   base   0.1  None     35   
724    image_to_text_R@1        UCM      35.0   64   base   0.1  None     35   
874    image_to_text_R@1        UCM      25.0  128   base   0.1  None     25   
721    image_to_text_R@1        UCM      30.0   64   base   0.1  None     30   
1057  image_to_text_R@10      RSICD      30.0   64   base   0.1  None     30   
1054  image_to_text_R@10      RSICD      25.0   64   base   0.1  None     25   
1060  image_to_text_R@10      RSICD      35.0   64   base   0.1  None     35   
1549  image_to_text_R@10     Sydney      30.0  128   base   0.1  None     30   
1393  image_to_text_R@10     Sydney      30.0   64   base   0.1  None     30   
1396  image_to_text_R@10     Sydney      35.0   64   base   0.1  None     35   
1732  image_to_text_R@10        UCM      35.0   64   base   0.1  None     35   
1888  image_to_text_R@10        UCM      35.0  128   base   0.1  None     35   
1885  image_to_text_R@10        UCM      30.0  128   base   0.1  None     30   
2062   image_to_text_R@5      RSICD      25.0   64   base   0.1  None     25   
2065   image_to_text_R@5      RSICD      30.0   64   base   0.1  None     30   
2068   image_to_text_R@5      RSICD      35.0   64   base   0.1  None     35   
2401   image_to_text_R@5     Sydney      30.0   64   base   0.1  None     30   
2557   image_to_text_R@5     Sydney      30.0  128   base   0.1  None     30   
2560   image_to_text_R@5     Sydney      35.0  128   base   0.1  None     35   
2896   image_to_text_R@5        UCM      35.0  128   base   0.1  None     35   
2740   image_to_text_R@5        UCM      35.0   64   base   0.1  None     35   
2893   image_to_text_R@5        UCM      30.0  128   base   0.1  None     30   
3070   text_to_image_R@1      RSICD      25.0   64   base   0.1  None     25   
3073   text_to_image_R@1      RSICD      30.0   64   base   0.1  None     30   
3076   text_to_image_R@1      RSICD      35.0   64   base   0.1  None     35   
3562   text_to_image_R@1     Sydney      25.0  128   base   0.1  None     25   
3565   text_to_image_R@1     Sydney      30.0  128   base   0.1  None     30   
3568   text_to_image_R@1     Sydney      35.0  128   base   0.1  None     35   
3745   text_to_image_R@1        UCM      30.0   64   base   0.1  None     30   
3901   text_to_image_R@1        UCM      30.0  128   base   0.1  None     30   
3904   text_to_image_R@1        UCM      35.0  128   base   0.1  None     35   
4078  text_to_image_R@10      RSICD      25.0   64   base   0.1  None     25   
4084  text_to_image_R@10      RSICD      35.0   64   base   0.1  None     35   
4081  text_to_image_R@10      RSICD      30.0   64   base   0.1  None     30   
4576  text_to_image_R@10     Sydney      35.0  128   base   0.1  None     35   
4420  text_to_image_R@10     Sydney      35.0   64   base   0.1  None     35   
4417  text_to_image_R@10     Sydney      30.0   64   base   0.1  None     30   
4912  text_to_image_R@10        UCM      35.0  128   base   0.1  None     35   
4753  text_to_image_R@10        UCM      30.0   64   base   0.1  None     30   
4756  text_to_image_R@10        UCM      35.0   64   base   0.1  None     35   
5086   text_to_image_R@5      RSICD      25.0   64   base   0.1  None     25   
5092   text_to_image_R@5      RSICD      35.0   64   base   0.1  None     35   
5089   text_to_image_R@5      RSICD      30.0   64   base   0.1  None     30   
5581   text_to_ima

In [12]:
# Results for S-CLIP model (filtered a bit)
# & (df_results['lr']==5e-5)
df_S_CLIP = df_results[(df_results['AL.iter'].isna()) & (df_results['method'] == 'ours') & (df_results['PL'].str.contains('ot.'))]
df_PL_soft = df_results[(df_results['AL.iter'].isna()) & (df_results['method'] == 'ours') & (df_results['PL'].str.contains('soft.'))]
df_PL_hard = df_results[(df_results['AL.iter'].isna()) & (df_results['method'] == 'ours') & (df_results['PL'].str.contains('hard.'))]

In [13]:
full_display(df_S_CLIP.groupby(['metric', 'dataset']).tail(3))

metric    dataset AL.epochs   bs method ratio        PL  \
                                                                            
265    image_to_text_R@1      RSICD      25.0  128   ours   0.1  ot.image   
268    image_to_text_R@1      RSICD      30.0  128   ours   0.1  ot.image   
271    image_to_text_R@1      RSICD      35.0  128   ours   0.1  ot.image   
595    image_to_text_R@1     Sydney      15.0  128   ours   0.1  ot.image   
604    image_to_text_R@1     Sydney      30.0  128   ours   0.1  ot.image   
607    image_to_text_R@1     Sydney      35.0  128   ours   0.1  ot.image   
937    image_to_text_R@1        UCM      25.0  128   ours   0.1  ot.image   
940    image_to_text_R@1        UCM      30.0  128   ours   0.1  ot.image   
943    image_to_text_R@1        UCM      35.0  128   ours   0.1  ot.image   
1273  image_to_text_R@10      RSICD      25.0  128   ours   0.1  ot.image   
1279  image_to_text_R@10      RSICD      35.0  128   ours   0.1  ot.image   
1276  image_to_text_R@10      RSICD      30.0  128   ours   0.1  ot.image   
1609  image_to_text_R@10     Sydney      25.0  128   ours   0.1  ot.image   
1612  image_to_text_R@10     Sydney      30.0  128   ours   0.1  ot.image   
1615  image_to_text_R@10     Sydney      35.0  128   ours   0.1  ot.image   
1804  image_to_text_R@10        UCM      35.0   64   ours   0.1  ot.image   
1948  image_to_text_R@10        UCM      30.0  128   ours   0.1  ot.image   
1951  image_to_text_R@10        UCM      35.0  128   ours   0.1  ot.image   
2281   image_to_text_R@5      RSICD      25.0  128   ours   0.1  ot.image   
2284   image_to_text_R@5      RSICD      30.0  128   ours   0.1  ot.image   
2287   image_to_text_R@5      RSICD      35.0  128   ours   0.1  ot.image   
2473   image_to_text_R@5     Sydney      30.0   64   ours   0.1  ot.image   
2620   image_to_text_R@5     Sydney      30.0  128   ours   0.1  ot.image   
2623   image_to_text_R@5     Sydney      35.0  128   ours   0.1  ot.image   
2959   image_to_text_R@5        UCM      35.0  128   ours   0.1  ot.image   
2956   image_to_text_R@5        UCM      30.0  128   ours   0.1  ot.image   
2953   image_to_text_R@5        UCM      25.0  128   ours   0.1  ot.image   
3292   text_to_image_R@1      RSICD      30.0  128   ours   0.1  ot.image   
3295   text_to_image_R@1      RSICD      35.0  128   ours   0.1  ot.image   
3148   text_to_image_R@1      RSICD      35.0   64   ours   0.1  ot.image   
3631   text_to_image_R@1     Sydney      35.0  128   ours   0.1  ot.image   
3628   text_to_image_R@1     Sydney      30.0  128   ours   0.1  ot.image   
3625   text_to_image_R@1     Sydney      25.0  128   ours   0.1  ot.image   
3961   text_to_image_R@1        UCM      25.0  128   ours   0.1  ot.image   
3964   text_to_image_R@1        UCM      30.0  128   ours   0.1  ot.image   
3967   text_to_image_R@1        UCM      35.0  128   ours   0.1  ot.image   
4297  text_to_image_R@10      RSICD      25.0  128   ours   0.1  ot.image   
4300  text_to_image_R@10      RSICD      30.0  128   ours   0.1  ot.image   
4303  text_to_image_R@10      RSICD      35.0  128   ours   0.1  ot.image   
4633  text_to_image_R@10     Sydney      25.0  128   ours   0.1  ot.image   
4636  text_to_image_R@10     Sydney      30.0  128   ours   0.1  ot.image   
4639  text_to_image_R@10     Sydney      35.0  128   ours   0.1  ot.image   
4972  text_to_image_R@10        UCM      30.0  128   ours   0.1  ot.image   
4828  text_to_image_R@10        UCM      35.0   64   ours   0.1  ot.image   
4975  text_to_image_R@10        UCM      35.0  128   ours   0.1  ot.image   
5305   text_to_image_R@5      RSICD      25.0  128   ours   0.1  ot.image   
5311   text_to_image_R@5      RSICD      35.0  128   ours   0.1  ot.image   
5308   text_to_image_R@5      RSICD      30.0  128   ours   0.1  ot.image   
5641   text_to_image_R@5     Sydney      25.0  128   ours   0.1  ot.image   
5644   text_to_image_R@5     Sydney      30.0  128   ours   0.1  ot.image   
5647   text_to

In [14]:
full_display(df_PL_soft.groupby(['metric', 'dataset']).tail(3))

metric    dataset AL.epochs   bs method ratio          PL  \
                                                                              
184    image_to_text_R@1      RSICD      30.0   64   ours   0.1   soft.text   
187    image_to_text_R@1      RSICD      35.0   64   ours   0.1   soft.text   
310    image_to_text_R@1      RSICD      30.0  128   ours   0.1  soft.image   
643    image_to_text_R@1     Sydney      25.0  128   ours   0.1  soft.image   
520    image_to_text_R@1     Sydney      30.0   64   ours   0.1   soft.text   
523    image_to_text_R@1     Sydney      35.0   64   ours   0.1   soft.text   
859    image_to_text_R@1        UCM      35.0   64   ours   0.1   soft.text   
1000   image_to_text_R@1        UCM      25.0  128   ours   0.1   soft.text   
1006   image_to_text_R@1        UCM      35.0  128   ours   0.1   soft.text   
1342  image_to_text_R@10      RSICD      35.0  128   ours   0.1   soft.text   
1195  image_to_text_R@10      RSICD      35.0   64   ours   0.1   soft.text   
1192  image_to_text_R@10      RSICD      30.0   64   ours   0.1   soft.text   
1675  image_to_text_R@10     Sydney      30.0  128   ours   0.1   soft.text   
1678  image_to_text_R@10     Sydney      35.0  128   ours   0.1   soft.text   
1525  image_to_text_R@10     Sydney      25.0   64   ours   0.1   soft.text   
1990  image_to_text_R@10        UCM      30.0  128   ours   0.1  soft.image   
1867  image_to_text_R@10        UCM      35.0   64   ours   0.1   soft.text   
1864  image_to_text_R@10        UCM      30.0   64   ours   0.1   soft.text   
2329   image_to_text_R@5      RSICD      35.0  128   ours   0.1  soft.image   
2200   image_to_text_R@5      RSICD      30.0   64   ours   0.1   soft.text   
2203   image_to_text_R@5      RSICD      35.0   64   ours   0.1   soft.text   
2680   image_to_text_R@5     Sydney      25.0  128   ours   0.1   soft.text   
2683   image_to_text_R@5     Sydney      30.0  128   ours   0.1   soft.text   
2686   image_to_text_R@5     Sydney      35.0  128   ours   0.1   soft.text   
2872   image_to_text_R@5        UCM      30.0   64   ours   0.1   soft.text   
3019   image_to_text_R@5        UCM      30.0  128   ours   0.1   soft.text   
2875   image_to_text_R@5        UCM      35.0   64   ours   0.1   soft.text   
3211   text_to_image_R@1      RSICD      35.0   64   ours   0.1   soft.text   
3208   text_to_image_R@1      RSICD      30.0   64   ours   0.1   soft.text   
3355   text_to_image_R@1      RSICD      30.0  128   ours   0.1   soft.text   
3694   text_to_image_R@1     Sydney      35.0  128   ours   0.1   soft.text   
3691   text_to_image_R@1     Sydney      30.0  128   ours   0.1   soft.text   
3670   text_to_image_R@1     Sydney      30.0  128   ours   0.1  soft.image   
4021   text_to_image_R@1        UCM      20.0  128   ours   0.1   soft.text   
4006   text_to_image_R@1        UCM      30.0  128   ours   0.1  soft.image   
4009   text_to_image_R@1        UCM      35.0  128   ours   0.1  soft.image   
4213  text_to_image_R@10      RSICD      25.0   64   ours   0.1   soft.text   
4216  text_to_image_R@10      RSICD      30.0   64   ours   0.1   soft.text   
4219  text_to_image_R@10      RSICD      35.0   64   ours   0.1   soft.text   
4699  text_to_image_R@10     Sydney      30.0  128   ours   0.1   soft.text   
4702  text_to_image_R@10     Sydney      35.0  128   ours   0.1   soft.text   
4681  text_to_image_R@10     Sydney      35.0  128   ours   0.1  soft.image   
5017  text_to_image_R@10        UCM      35.0  128   ours   0.1  soft.image   
4888  text_to_image_R@10        UCM      30.0   64   ours   0.1   soft.text   
4891  text_to_image_R@10        UCM      35.0   64   ours   0.1   soft.text   
5221   text_to_image_R@5      RSICD      25.0   64   ours   0.1   soft.text   
5224   text_to_image_R@5      RSICD      30.0   64   ours   0.1   soft.text   
5227   text_to_image_R@5      RSICD      35.0   64   ours   0.1   soft.text   
5683   text_to_image_R@5     Sydney      25.0  128   ours   0.1  sof

In [15]:
full_display(df_PL_hard.groupby(['metric', 'dataset']).tail(3))

metric    dataset AL.epochs   bs method ratio          PL  \
                                                                              
97     image_to_text_R@1      RSICD      25.0   64   ours   0.1   hard.text   
250    image_to_text_R@1      RSICD      35.0  128   ours   0.1   hard.text   
103    image_to_text_R@1      RSICD      35.0   64   ours   0.1   hard.text   
433    image_to_text_R@1     Sydney      25.0   64   ours   0.1   hard.text   
583    image_to_text_R@1     Sydney      30.0  128   ours   0.1   hard.text   
586    image_to_text_R@1     Sydney      35.0  128   ours   0.1   hard.text   
898    image_to_text_R@1        UCM      30.0  128   ours   0.1  hard.image   
901    image_to_text_R@1        UCM      35.0  128   ours   0.1  hard.image   
919    image_to_text_R@1        UCM      30.0  128   ours   0.1   hard.text   
1258  image_to_text_R@10      RSICD      35.0  128   ours   0.1   hard.text   
1108  image_to_text_R@10      RSICD      30.0   64   ours   0.1   hard.text   
1111  image_to_text_R@10      RSICD      35.0   64   ours   0.1   hard.text   
1444  image_to_text_R@10     Sydney      30.0   64   ours   0.1   hard.text   
1447  image_to_text_R@10     Sydney      35.0   64   ours   0.1   hard.text   
1594  image_to_text_R@10     Sydney      35.0  128   ours   0.1   hard.text   
1759  image_to_text_R@10        UCM      30.0   64   ours   0.1  hard.image   
1783  image_to_text_R@10        UCM      35.0   64   ours   0.1   hard.text   
1780  image_to_text_R@10        UCM      30.0   64   ours   0.1   hard.text   
2119   image_to_text_R@5      RSICD      35.0   64   ours   0.1   hard.text   
2113   image_to_text_R@5      RSICD      25.0   64   ours   0.1   hard.text   
2116   image_to_text_R@5      RSICD      30.0   64   ours   0.1   hard.text   
2449   image_to_text_R@5     Sydney      25.0   64   ours   0.1   hard.text   
2599   image_to_text_R@5     Sydney      30.0  128   ours   0.1   hard.text   
2602   image_to_text_R@5     Sydney      35.0  128   ours   0.1   hard.text   
2938   image_to_text_R@5        UCM      35.0  128   ours   0.1   hard.text   
2770   image_to_text_R@5        UCM      35.0   64   ours   0.1  hard.image   
2935   image_to_text_R@5        UCM      30.0  128   ours   0.1   hard.text   
3121   text_to_image_R@1      RSICD      25.0   64   ours   0.1   hard.text   
3124   text_to_image_R@1      RSICD      30.0   64   ours   0.1   hard.text   
3127   text_to_image_R@1      RSICD      35.0   64   ours   0.1   hard.text   
3604   text_to_image_R@1     Sydney      25.0  128   ours   0.1   hard.text   
3610   text_to_image_R@1     Sydney      35.0  128   ours   0.1   hard.text   
3607   text_to_image_R@1     Sydney      30.0  128   ours   0.1   hard.text   
3796   text_to_image_R@1        UCM      30.0   64   ours   0.1   hard.text   
3946   text_to_image_R@1        UCM      35.0  128   ours   0.1   hard.text   
3778   text_to_image_R@1        UCM      35.0   64   ours   0.1  hard.image   
4129  text_to_image_R@10      RSICD      25.0   64   ours   0.1   hard.text   
4132  text_to_image_R@10      RSICD      30.0   64   ours   0.1   hard.text   
4135  text_to_image_R@10      RSICD      35.0   64   ours   0.1   hard.text   
4468  text_to_image_R@10     Sydney      30.0   64   ours   0.1   hard.text   
4618  text_to_image_R@10     Sydney      35.0  128   ours   0.1   hard.text   
4615  text_to_image_R@10     Sydney      30.0  128   ours   0.1   hard.text   
4804  text_to_image_R@10        UCM      30.0   64   ours   0.1   hard.text   
4801  text_to_image_R@10        UCM      25.0   64   ours   0.1   hard.text   
4807  text_to_image_R@10        UCM      35.0   64   ours   0.1   hard.text   
5137   text_to_image_R@5      RSICD      25.0   64   ours   0.1   hard.text   
5140   text_to_image_R@5      RSICD      30.0   64   ours   0.1   hard.text   
5143   text_to_image_R@5      RSICD      35.0   64   ours   0.1   hard.text   
5473   text_to_image_R@5     Sydney      25.0   64   ours   0.1   ha